In [1]:
import os

In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [ ]:
load_dotenv()


In [ ]:
key=os.getenv("OPENAI_API_KEY")
print(key)

In [ ]:
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.7)
llm

In [ ]:
with open("E:\GenAI\GenAI\Response.json","r") as f:
    response_json=json.load(f)
response_json

In [7]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text,it is your job to create a quiz \
of{number} multiple choice questions for {subject} students in {tone}\
Make sure the questions are not repeated and check all the questions to be conforming 
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}  
"""

In [8]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","respone_json"],
    template=TEMPLATE
)

In [ ]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [10]:
Template2="""
You are an expert english grammarian and writer.Given a Multiple Choice Quiz for {subject}.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz.Only use at max 50 words for complexity analysis.
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student ability
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:

"""

In [11]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=Template2
)

In [12]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [13]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],output_variables=["quiz","review"],verbose=True)

In [ ]:
response_json

In [ ]:
json.dumps(response_json)

In [ ]:
PATH = "E:/GenAI/GenAI/data.txt"

# 'utf-8' encoding ka use karo
try:
    with open(PATH, "r", encoding="utf-8") as file:
        Text = file.read()
except UnicodeDecodeError:
    # Agar 'utf-8' fail ho jaye, toh 'latin-1' use karo
    with open(PATH, "r", encoding="latin-1") as file:
        Text = file.read()

print(Text)

In [17]:
Text
Number=5
SUBJECT="Physics"
Tone="Simple"
response_json=response_json

In [ ]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
        "text":Text,
        "number":Number,
        "subject":SUBJECT,
        "tone":Tone,
        "response_json":json.dumps(response_json)
        }
    )

In [ ]:
response

In [25]:
quiz_res=response.get("quiz")

In [ ]:
print(repr(quiz_res))

In [ ]:
if quiz_res.startswith("### RESPONSE_JSON"):
    quiz_res = quiz_res[len("### RESPONSE_JSON\n"):]

# Parse the JSON
try:
    quiz_res_json = json.loads(quiz_res)
    print("Parsed JSON:", quiz_res_json)
except json.JSONDecodeError as e:
    print("Failed to parse JSON:", e)

In [ ]:
type(quiz_res)

In [33]:
quiz_res=json.loads(quiz_res)

In [ ]:
type(quiz_res)

In [ ]:
quiz_res

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completions Tokens:{cb.completion_tokens}")
print(f"Cost Tokens:{cb.total_cost}")

In [45]:
quiz_table_data=[]
for key,value in quiz_res.items():
    mcq=value["mcq"]
    options="|".join (
        [
            f"{option}:{option_value}"
            for option,option_value in value["options"].items()
        ]
    )
    correct=value["correct"]
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"correct":correct})

In [ ]:
type(quiz_table_data)

In [47]:
df=pd.DataFrame(quiz_table_data)

In [ ]:
df

In [ ]:
df["Choices"]